In [7]:
from classify import data_pipeline, eco_selector
from resources.get_data import get_current_data
from resources.setup import get_setup
import pandas as pd

In [6]:
def validator(df):
    df = df.drop_duplicates()
    df = df[~df['text'].isna()]
    if 'vectorized' in df.columns.values:
        df = df.drop(columns=['vectorized'])
    if df.index.duplicated().any():
        print("Duplicated index!!!")
        df = df.reset_index()
        df = df.drop(columns = ['id'])
        df = df.rename(columns = {'index': 'id'})
    print(f"Found {len(df)} files")
    return df

# Big corpuses

## Rzepa

In [ ]:
corp = 'rzepa'
df_rest, df_eco = data_pipeline(corp)
df_final = pd.concat([df_eco, eco_selector(df_rest)])
df_final = validator(df_final)
df_final.to_csv(f"eco_{corp}.csv")

## Wyborcza

In [ ]:
corp = 'wyborcza'
df_rest, df_eco = data_pipeline(corp)
df_final = pd.concat([df_eco, eco_selector(df_rest)])
df_final = validator(df_final)
df_final.to_csv(f"eco_{corp}.csv")

In [ ]:
df_final = validator(df_final)
df_final.to_csv(f"eco_{corp}.csv")

## Gazeta Polska Codziennie

In [3]:
corp = 'gpc'
df_rest, df_eco = data_pipeline(corp)
df_eco = pd.concat([df_eco, eco_selector(df_rest, False)])
df_eco = validator(df_eco)
df_eco.to_csv(f"eco_{corp}.csv")

{1: 'rzepa', 2: 'gpc', 3: 'newsweek', 4: 'wprost', 5: 'dorzeczy', 6: 'polityka', 7: 'wyborcza', 8: 'wpolityce'}
gpc
Configuration finished
Dataset of 7471 samples
label
1    4128
0    3343
Name: count, dtype: int64
Preprocessing ended
Accuracy of model 0.9986631016042781


100%|██████████| 24/24 [00:29<00:00,  1.24s/it]


11504 1149
Duplicated index!!!
Found 3278 files


## Polityka

In [4]:
corp = 'polityka'
df_rest, df_eco = data_pipeline(corp)
df_eco = pd.concat([df_eco, eco_selector(df_rest)])
df_eco = validator(df_eco)
df_eco.to_csv(f"eco_{corp}.csv")

{1: 'rzepa', 2: 'gpc', 3: 'newsweek', 4: 'wprost', 5: 'dorzeczy', 6: 'polityka', 7: 'wyborcza', 8: 'wpolityce'}
polityka
Configuration finished
Dataset of 11849 samples
label
1    8700
0    3149
Name: count, dtype: int64
Preprocessing ended
Accuracy of model 0.9873417721518988


100%|██████████| 4/4 [00:12<00:00,  3.16s/it]


1909 220
Found 754 files


# Small Corpuses

## Dorzeczy

In [5]:
from datetime import datetime

corp = 'dorzeczy'
df_rest, df_eco = data_pipeline(corp)
df_eco = pd.concat([df_eco, eco_selector(df_rest)])
df_eco = df_eco.reset_index().drop(columns='id').rename(columns={'Unnamed: 0': 'id'}).set_index('id')
df_eco = df_eco[df_eco['date'] < datetime(2023, 1, 1)]
df_eco = validator(df_eco)
df_eco.to_csv(f"eco_{corp}.csv")

{1: 'rzepa', 2: 'gpc', 3: 'newsweek', 4: 'wprost', 5: 'dorzeczy', 6: 'polityka', 7: 'wyborcza', 8: 'wpolityce'}
dorzeczy
Configuration finished
Dataset of 11342 samples
label
1    8400
0    2942
Name: count, dtype: int64
Preprocessing ended
Accuracy of model 0.9947136563876652


100%|██████████| 2/2 [00:04<00:00,  2.44s/it]


696 76
Found 242 files


## Wprost

In [9]:
corp = 'wprost'
df_rest, df_eco = data_pipeline(corp)
df_eco = pd.concat([df_eco, eco_selector(df_rest)])
df_eco = validator(df_eco)
df_eco.to_csv(f"eco_{corp}.csv")

{1: 'rzepa', 2: 'gpc', 3: 'newsweek', 4: 'wprost', 5: 'dorzeczy', 6: 'polityka', 7: 'wyborcza', 8: 'wpolityce'}
wprost
Configuration finished
Dataset of 5279 samples
label
0    2911
1    2368
Name: count, dtype: int64
Preprocessing ended
Accuracy of model 0.9867424242424242


100%|██████████| 2/2 [00:04<00:00,  2.42s/it]


550 34
Duplicated index!!!
Found 402 files


In [6]:
df_eco = pd.concat([df_eco, df_rest.drop(index=2260)])
df_eco = validator(df_eco)
df_eco.to_csv(f"eco_{corp}.csv")

Duplicated index!!!
Found 342 files


## Newsweek

In [10]:
corp = 'newsweek'
df_rest, df_eco = data_pipeline(corp)
df_eco = pd.concat([df_eco, eco_selector(df_rest)])
df_eco = validator(df_eco)
df_eco.to_csv(f"eco_{corp}.csv")

{1: 'rzepa', 2: 'gpc', 3: 'newsweek', 4: 'wprost', 5: 'dorzeczy', 6: 'polityka', 7: 'wyborcza', 8: 'wpolityce'}
newsweek
{1: 'rzepa', 2: 'gpc', 3: 'newsweek', 4: 'wprost', 5: 'dorzeczy', 6: 'polityka', 7: 'wyborcza', 8: 'wpolityce'}
newsweek
Configuration finished
Dataset of 11417 samples
label
1    8452
0    2965
Name: count, dtype: int64
Preprocessing ended
Accuracy of model 0.989492119089317


100%|██████████| 2/2 [00:05<00:00,  2.89s/it]


933 104
Duplicated index!!!
Found 390 files


## wPolityce

In [7]:
corp = 'wpolityce'
df_rest, df_eco = data_pipeline(corp)
df_eco = pd.concat([df_eco, eco_selector(df_rest)])
df_eco = validator(df_eco)
df_eco.to_csv(f"eco_{corp}.csv")

{1: 'rzepa', 2: 'gpc', 3: 'newsweek', 4: 'wprost', 5: 'dorzeczy', 6: 'polityka', 7: 'wyborcza', 8: 'wpolityce'}
wpolityce
Configuration finished
Dataset of 5298 samples
label
0    2962
1    2336
Name: count, dtype: int64
Preprocessing ended
Accuracy of model 0.9924528301886792


100%|██████████| 3/3 [00:23<00:00,  7.90s/it]


1362 76
Found 413 files


# Corpus

In [10]:
import os
corps = [file for file in os.listdir() if file.startswith("eco")]
print(corps)

['eco_gpc.csv', 'eco_wyborcza.csv', 'eco_rzepa.csv', 'eco_polityka.csv', 'eco_dorzeczy.csv', 'eco_wprost.csv', 'eco_newsweek.csv', 'eco_wpolityce.csv']


In [11]:
data = []
for corp in corps:
    print(corp)
    data.append(pd.read_csv(corp, usecols=['clean_text']))

df = pd.concat(data)

eco_gpc.csv


eco_wyborcza.csv
eco_rzepa.csv
eco_polityka.csv
eco_dorzeczy.csv
eco_wprost.csv
eco_newsweek.csv
eco_wpolityce.csv


In [14]:
from files.ngram.ngrams import ngram_counter
df = df[~df['clean_text'].isna()]
for ngram in range(1, 4):
    df_ngram = ngram_counter(ngram, df).iloc[:100]
    df_ngram.to_csv(f"most_common_{ngram}-gram.csv")